# Mobilinkd M17 Setup Guide

This is a guide on how to properly configure and use the Mobilinkd TNC3
for M17 voice and data communication.

This guide assumes that you will be using the TNC3 with a suitable
9600-baud capable radio, and that you have the proper cable to
connect the TNC3 to the radio.  It also assumes that you will be
using the [M17-KISS-HT](https://github.com/mobilinkd/m17-kiss-ht)
Android app for voice communications.

The guide assumes familiarity with GNU Radio for the M17 receive
tests.  And it assumes that you can side-load apps to your Android
device.

This is very much a guide for the experienced experimenter, and not
a step-by-step guide designed for beginners.

I will gladly accept pull requests for added detail for any of the
sections below, or for setup hints for specific radio models.

## Required Tools

 1. An RTL-SDR (TCXO-based, or well-characterized) with antenna.
 1. A Pluto SDR with accurate timebase (or other suitable transmit-capable SDR)
 1. A Linux-based computer with working sound system
 1. The `rtl-sdr` package installed, which contains `/usr/bin/rtl_fm`.
 1. [m17-cxx-demod](https://github.com/mobilinkd/m17-cxx-demod) built for Linux
 1. GQRX or other GUI-based software for viewing the spectrum by an RTL-SDR
 1. A working GNU Radio installation with gr-iio (or sink for your SDR transmitter)
 1. Android device running a recent version of Android (10 or greater)
 1. The Mobilinkd Config App installed on the Android device
 1. M17-KISS-HT installed on the Android device (must be side-loaded)
 1. A USB-OTG adapter for the Android device, and an USB cable to connect to the TNC
 1. A dummy load
 
Please note that both SDRs will require accurate reference clocks.
4-FSK requires reasonably accurate transmit and receive frequencies to
be decoded properly.  If your SDR has a 20ppm clock, you can be off by
8kHz on 70cm.  It will be impossible to tune the system properly with
such an error.  The maximum error that will work is about 500Hz, and
less than 100Hz is preferred. That's a 1ppm error, max, at 70cm.

We will be connecting to the TNC via USB-OTG rather than over BLE
as it been considerably more reliable.  The BLE connection has been
unreliable on certain phones and Android versions.

### Recommended Tools

 1. RF filters for the transmit SDR
 1. RF attenuators 20-40dB
 1. Spectrum Analyzer
 1. Cables and adapters that allow connecting the SDR to
    the radio through an attenuator.
 
## Overview

We will be performing the following steps:

 1. Configure the TX deviation of the TNC/radio pair
 1. Configure the TX polarity of the TNC/radio pair
 1. Run a TX BERT (bit error rate test)
 1. Set up M17-KISS-HT
 1. Configure the RX polarity of the TNC/radio pair
 1. Run an RX BERT
 1. Transmit and receive voice

## Resources

 * TNC3 Firmware: https://github.com/mobilinkd/tnc3-firmware/releases
 * Mobilinkd Config App: https://play.google.com/store/apps/details?id=com.mobilinkd.tncconfig
 * M17 KISS HT Android App: https://github.com/mobilinkd/m17-kiss-ht/releases
 * M17 C++ Demodulator (m17-cxx-demod): https://github.com/mobilinkd/m17-cxx-demod/

## Prep Work

All of the detailed setup steps outlined in the sections below assume that
this prep work has been completed.

 * Connect the radio to a suitable dummy load.
 * Install the M17 firmware (currently version 2.4.1) on the TNC3.
 * Install the Mobilinkd Config app on the Android device.
 * Side-load the M17 KISS HT app on the Android device.
 * Pair the Mobilinkd TNC3 with the Android Device.  Leave the TNC turned on.
 * Connect the TNC3 to the radio.
 * Turn on the radio and set it to the desired test frequency.
 * Connect the RTL-SDR with an antenna to the computer.
 * Connect the Pluto (with an antenna connected to the TX port) to the computer.
 * Install GQRX and GNU Radio if not already installed.


### Configure the TNC3 for M17

Open the Mobilinkd Config app, connect to the TNC, and go to *Modem Settings*.

Select *M17 4-FSK* as the modem type.

Select *Close*.


<table>
  <tr>
    <td> <img src="images/m17-modem-settings.png" alt="TNC3 modem settings for M17" style="width: 300px;"/> </td>
  </tr>
</table>


Select *Save Settings*

### Install and Build m17-cxx-demod

Install the required libraries for m17-cxx-demod: boost-devel and codec2-devel.
These may be named differently on your Linux platform.  `m17-cxx-demod` requires
the `codec2` library and the `Boost program-options` library.

Clone the m17-cxx-demod repo and build the M17 C++ tools.

    git clone https://github.com/mobilinkd/m17-cxx-demod.git
    cd m17-cxx-demod
    mkdir build
    cd build
    cmake ..
    make
    make test

## When Transmitting...

Choose a test frequency that is unused and unlikely to interfere with
other amateur radio users.

You may have an obligation to announce your callsign on the frequency that
you are transmitting on.  If you are conducting open-air tests using an
antenna -- not via a dummy load or via direct coax connection between
radios -- you should announce at the start of the test, as frequently
as required by law, and when you are finished with the tests.

## Mobilinkd Config App

The *Audio Output Settings* of the Mobilinkd Config app was designed for
configuring 1200 baud AFSK audio levels.  There are tone settings for
1200Hz, 2200Hz, and Both.  Those settings are mapped as follows when
the TNC is in M17 mode:

 * 1200Hz -> 2400Hz (M17 Preamble)
 * 2200Hz -> 1000Hz (M17 Deviation Aid)
 * BOTH -> M17 BER Mode.

BER mode implements the M17 BER testing specification.

## Transmit Deviation

This section assumes that you have completed the [Prep Work](#Prep-Work) steps above.

**NOTE:** The preferred way to do this is with a spectrum analyzer instead of an
RTL-SDR and GQRX.  Use an appropriately sized attenuator between the radio and the
instrument.  Set the span to 50kHz and the RBW to 300Hz.  Follow the output
adjustment procedure below.

Start `GQRX` and tune to the desired frequency.  The examples below use
144.910MHz for the test frequency.

In the *Input Controls* tab, set the appropriate frequency correction factor
for your RTL-SDR.  If you are not using an accurate TCXO, it must be set very
close to the PPM offset of your RTL-SDR.  You need to be within 1PPM.

In the *Receiver Options* tab, select Narrow FM in the *Mode* setting.  Select
the "..." to the right of the *Mode* selection.  Set the maximum deviation to
2.5kHz and the Tau factor to 0.

In the *FFT Settings* tab, use the *Freq Zoom* slider to increase the zoom
factor to 10-20x.  You may need to adjust this later while testing to get the
best view.

Start the receiver using *Play* button.

Open the Mobilinkd Config app on the Android device, connect to the TNC,
then select *Audio Output Settings*.  Select the *2200Hz* for the *Output Tone*.
When in M17 mode, this will transmit a 1000Hz tone.  A 1000Hz tone will create
the first Bessel null at 2405Hz deviation.  This is very close to the value
needed for M17.

Press the *Transmit* button.  Observe the received spectrum in *GQRX*.

<table>
  <tr>
    <td> <img src="images/m17-audio-output.png" alt="TNC3 audio output for M17" style="width: 300px;"/> </td>
  </tr>
</table>

Adjust the output level until you see carrier (the middle peak) dip to its
lowest point.  It may require careful adjustment of the slider on the app
as it was not really designed for such small adjustments that are required
here.

<table>
  <tr>
    <td> <img src="images/m17-bessel-null.png" alt="GQRX Bessel null for M17" style="width: 600px;"/> </td>
  </tr>
</table>

After the output setting has been adjusted to the desired level, press the
*Transmit* button to stop transmitting.  Press *Close* to go back to the
main menu, then select *Save Settings*.

Press the *Play* button in *GQRX* to stop the receiver.  You can now exit *GQRX*.

## Configure TX Polarity

This section assumes that you have completed the [Prep Work](#Prep-Work) steps above. 

We are going to run rtl_fm and m17-cxx-demod to receive and decode M17 from the
TNC and radio.  The TNC will sent a BER (bit error-rate) test sequence.

Inside the `m17-cxx-demod/build` directory from earlier, run the following command:

    rtl_fm -F 9 -f 144.91M -s 18k | sox -t s16 -r 18k -c1 - -t raw - gain 9 rate -v -s 48k | ./apps/m17-demod -d -l > /dev/null

You may need to set a `"-p"` option value for `rtl_fm` to adjust for the
clock error on your RTL-SDR dongle.

Using a wide screen is best since it will output a wide diagnostics line.  If the
terminal window is too narrow, it will scroll endlessly and be hard to read.  A
width of 160 characters is about right.

 * Open the *Mobilinkd Config App* on the Android device.
 * Connect to the TNC.
 * Open *Audio Output Settings*.
 * Select *Both* for the *Output Tone*.  This will send a BER test sequence.

<table>
  <tr>
    <td> <img src="images/m17-tx-ber-test.png" alt="M17 BER test for TX Polarity" style="width: 300px;"/> </td>
  </tr>
</table>

Observe the results from the `m17-demod` program in the terminal window.  A
good result will look something like this:

<table>
  <tr>
    <td> <img src="images/m17-ber-test-2.png" alt="M17 BER test for TX Polarity (good)" style="width: 600px;"/> </td>
  </tr>
</table>

If, instead, you see something like the below result showing a high BER rate
(typically 80+), you likely need to change the polarity.

<table>
  <tr>
    <td> <img src="images/m17-ber-test-1.png" alt="M17 BER test for TX Polarity (bad)" style="width: 600px;"/> </td>
  </tr>
</table>

Stop transmitting, close the *Audio Output Settings* and open *Modem Settings*.
Enable the *TX Reverse Polarity* option.  Then repeat the BER test above.

<table>
  <tr>
    <td> <img src="images/m17-tx-reverse-polarity.png" alt="M17 setting TX Reverse Polarity" style="width: 300px;"/> </td>
  </tr>
</table>

Stop transmitting. If the results look good after changing the polarity, go
back to the main menu and press *Save Settings*.

If the results are still bad, stop check all of the frequency settings.  Verify
the PPM settings on the RTL-SDR.  Ensure that your radio's VCO is properly
calibrated and that it is transmitting and receiving at the proper frequency.

You can also reach out to #public-chat on the M17 Discord for help.

In the Mobilindk Config app, press the *Disconnect* button to disconnect from
the TNC.

## M17 KISS HT

This section assumes that you have completed the [Prep Work](#Prep-Work) steps
above, including side loading of the *M17 KISS HT* app.

 * Ensure that there is no existing connection to the TNC.
 * Connect the USB cable to the USB OTG adapter.
 * Plug the USB OTG adapter into the Android device.
 * Plug the USB cable into the Mobilinkd TNC.

You should be presented with a dialog to open the connection with an app.
Choose the *M17 HT* app.

<table>
  <tr>
    <td> <img src="images/m17-usb-connection.png" alt="M17 TNC USB Connection" style="width: 300px;"/> </td>
  </tr>
</table>

The first thing you need to do in order to use the app is to set your
callsign.  This can include a suffix designator.  Here you see that I
chose "WX9O-8" to distinguish this connection.

<table>
  <tr>
    <td> <img src="images/m17-kiss-ht-callsign.png" alt="M17 KISS HT Callsign" style="width: 300px;"/> </td>
  </tr>
</table>

### Voice Transmission

We are going to run the m17-cxx-demod program again, this time taking the
demodulated output and sending it to the computer's sound system.

    rtl_fm -F 9 -f 144.91M -s 18k | sox -t s16 -r 18k -c1 - -t raw - gain 9 rate -v -s 48k | ./apps/m17-demod -d -l | play -b 16 -r 8000 -c1 -t s16 -

This is the same command used for BER testing and setting up the TX polarity,
except we are now piping the output to the computer's audio output.

Press the transmit button on the M17 HT app.  A short press locks the transmit
button.  Another short press disengages the transmit button.  If instead you
hold the button for longer than 1/2 second, it will stop transmitting as soon
as you release the button.

Press the transmit button, speak, then release the transmit button.  It is
likely that you will hear a long delay before the received audio plays.  This
is because the pipeline used in the command above has high latency due to
the buffers in each stage of the pipeline.

You should hear your voice and see something like this in the terminal window.

<table>
  <tr>
    <td> <img src="images/m17-ht-voice-transmit.png" alt="M17 KISS Voice Transmit Test" style="width: 600px;"/> </td>
  </tr>
</table>

Notice that the callsign was decoded.

The long audio delay can be reduced somewhat by using `stdbuf`.  This makes the
command-line a bit unwieldy.

    stdbuf -o0 rtl_fm -F 9 -f 144.91M -s 18k | stdbuf -i0 -o0 sox --buffer 24 -t s16 -r 18k -c1 - -t raw - gain 9 rate -v -s 48k | stdbuf -i0 -o0 ./apps/m17-demod -d -l -b | stdbuf -i0  play --buffer 24 -q -b 16 -r 8000 -c1 -t s16 -

### M17 BER Receive Test

For this test you will need the `m17-gnuradio` repo cloned.

**NOTE:** If you are using an SDR other than an ADALM Pluto, please
verify that the output polarity is correct first by attempting to
receive it using the method outline in [Configure TX Polarity](#Configure-TX-Polarity).

Set up your Pluto SDR or other transmit-capable SDR.  Install any required
filters and transmit antenna.  If you can, connect the SDR to the radio
under test through a 30dB or 40dB attenuator.  Be very careful if you do
connect the SDR to the radio because it is very easy to damage the SDR
if the radio transmits.

Open *GNU Radio Companion* (GRC) and within that, open the `m17-bert.grc` file.
Adjust the output sink paramters (including the frequency) for the test.  You
may need to adjust the pathnam in the *File Source* block so that the `bert.bin` 
file can be found.  The path is typically relative to your HOME directory.

Once done, click the *Play* button on GRC. It should start transmitting a
BER test sequence similar to the one transmitted by the Mobilinkd Config app.

Connect the TNC to the Android device via the USB-OTG connection used earlier.
Once connected, the TNC should start receiving the BER test sequence.  It is
likely that you will see some error if you start receiving in the middle of
the test sequence.

<table>
  <tr>
    <td> <img src="images/m17-rx-ber-test.png" alt="M17 KISS BER Test" style="width: 300px;"/> </td>
  </tr>
</table>

If you do not see the app go into BER test mode, you will need to troubleshoot
the issue.

 1. Verify that the receive indicator on the TNC is green.  If not, check the
    transmit frequency and transmit power levels.  Check the radio's receive
    frequency is correct and that it is not set for a repeater split.
 1. If the receive indicator is green, disconnect the USB OTG cable from
    the Android device.  Open the Mobilinkd Config app. Connect to the TNC.
    Select *Modem Settings*, then enable `RX Reverse Polarity`. Press *Close*
    and then *Save Settings*.  Repeat the experiment.

### Voice Reception

The voice reception test is quite similar to the BER test above.  It is another
GNU Radio flowgraph that transmits a long-form English language voice program
for amateur radio.

This assumes the ADALM Pluto, radio and TNC are configured the same as with
the BER Receive Test.  If it is not, follow the setup steps in that section.

Open *GNU Radio Companion* (GRC) and within that, open the `m17-arn.grc` file.
Adjust the output sink paramters (including the frequency) for the test.  You
may need to adjust the pathnam in the *File Source* block so that the `Report2288.bin` 
file can be found.  The path is typically relative to your HOME directory.

Connect the TNC to the Android device via the USB-OTG connection used earlier.
This should start the M17 HT app.

Click the *Play* button on GRC. It will start transmitting a voice program,
which you should hear being received on the Android device.

<table>
  <tr>
    <td> <img src="images/m17-rx-voice-test.png" alt="M17 KISS HT Voice Receive Test" style="width: 300px;"/> </td>
  </tr>
</table>

## M17 on Yaesu FT-991

This section outlines set up steps and some issues with the Yeasu FT-991.

### Radio Setup Steps

Press the *Menu* button and adjust the following settings to use the packet data port for M17:

 * Set `76 FM PKT PTT SELECT` to `DAKY`
 * Set `77 FM PKT PORT SELECT` to `DATA`
 * Set `78 FM PKT TX GAIN` to `64`
 * Set `79 FM PKT MODE` to `9600`
 
Press the *Menu* button again to go back to the main screen.

Press the *Mode* button and select `DATA-FM`.  Press *Mode* again to go back to the main screen.

### TNC Setup Steps

Enable the `TX Reverse Polarity` option in the modem settings for this radio.

### Known Issues

On my unit, the frequency stability when it first starts transmitting is rather poor.

<table>
  <tr>
    <td> <img src="images/wobbly-freq-accuracy.png" alt="FT-991 wobbly frequency" style="width: 600px;"/> </td>
  </tr>
</table>

Note the wobbly lines at the start of the transmission.  This lasts for a few
seconds.  It causes a burst of bit errors at the start of the transmission.